In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from scipy.signal import resample
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import tensorflow as tf
from sklearn.preprocessing import StandardScaler

sensor_train = pd.read_csv('../dataset/sensor_train_final.csv')
sensor_test = pd.read_csv('../dataset/sensor_test_final.csv')
y = sensor_train.groupby('fragment_id')['behavior_id'].min()

In [ ]:
src_names = ['time_point','acc_x', 'acc_y', 'acc_z', 'acc_xg', 'acc_yg', 'acc_zg', 'acc', 'acc_g']

def handle_features(data):
    data['acc'] = (data.acc_x ** 2 + data.acc_y ** 2 + data.acc_z ** 2) ** 0.5
    data['acc_g'] = (data.acc_xg ** 2 + data.acc_yg ** 2 + data.acc_zg ** 2) ** 0.5
    return data

# 重复补齐
def handle_mats(grouped_data):
    mats = [i.values for i in grouped_data]
    max_len = 60
    for i in range(len(mats)):
        # repeated padding
        origin_mat = np.copy(mats[i])
        while(True):
            padding_size = max_len - len(mats[i])
            if padding_size > len(mats[i]):
                mats[i] = np.r_[mats[i], origin_mat]
            else:
                break
        # regular padding
        if len(mats[i]) < max_len:
            padding_size = max_len - len(mats[i])
            mats[i] = np.r_[mats[i], np.zeros([padding_size, mats[i].shape[-1]])]
        else:
            mats[i] = mats[i][:max_len]
        mats[i] = mats[i][np.newaxis, :, :]

    return np.concatenate(mats, axis=0)

train_data = handle_features(sensor_train)
test_data = handle_features(sensor_test)

scaler = StandardScaler()
train_data[src_names] = scaler.fit_transform(train_data[src_names])
test_data[src_names] = scaler.transform(test_data[src_names])

train_data_grouped = [i.drop(columns='fragment_id') for _, i in train_data.groupby('fragment_id')]
train_labels = np.array([int(i.iloc[0]['behavior_id']) for i in train_data_grouped])
test_data_grouped = [i.drop(columns='fragment_id') for _, i in test_data.groupby('fragment_id')]

for i in range(len(train_data_grouped)):
    train_data_grouped[i].drop(columns='behavior_id', inplace=True)

x = handle_mats(train_data_grouped)
t = handle_mats(test_data_grouped)
x = x.reshape((15000, 60, 9, 1))
t = t.reshape((16000, 60, 9, 1))

x1 = (x[:,:,0,:]).reshape((15000, 60, 1, 1))
x = x[:,:,1:9,:].reshape((15000, 60, 4, 2))
t1 = (t[:,:,0,:]).reshape((16000, 60, 1, 1))
t = t[:,:,1:9,:].reshape((16000, 60, 4, 2))

In [ ]:
def Net():
    input1 = Input(shape=(60, 4, 2),name='input1')
    input2 = Input(shape=(60, 1, 1),name='input2')
    #####################
    ###################
    X3 = Conv2D(filters=64,kernel_size=(3, 1),activation='relu',padding='same',)(input2)
    X3 = Conv2D(filters=8,kernel_size=(3, 1),activation='relu',padding='same',)(X3)
    X3 = concatenate([MaxPooling2D(pool_size=(2, 1))(X3),AveragePooling2D(pool_size=(2, 1))(X3)])
    X3 = Dropout(0.2)(X3)
    ####################
    X4 = Conv2D(filters=64,kernel_size=(5, 1),activation='relu',padding='same',)(input2)
    X4 = Conv2D(filters=8,kernel_size=(5, 1),activation='relu',padding='same',)(X4)
    X4 = concatenate([MaxPooling2D(pool_size=(2, 1))(X4),AveragePooling2D(pool_size=(2, 1))(X4)])
    X4 = Dropout(0.2)(X4)
    #####################
    X5 = Conv2D(filters=64,kernel_size=(7, 1),activation='relu',padding='same',)(input2)
    X5 = Conv2D(filters=8,kernel_size=(7, 1),activation='relu',padding='same',)(X5)
    X5 = concatenate([MaxPooling2D(pool_size=(2, 1))(X5),AveragePooling2D(pool_size=(2, 1))(X5)])
    X5 = Dropout(0.2)(X5)
    #####################
    X0 = Conv2D(filters=128,kernel_size=(3, 1),activation='relu',padding='same',)(input1+tf.reshape(X3,(-1,60,4,2)))
    X0 = Conv2D(filters=128,kernel_size=(3, 3),activation='relu',padding='same',)(X0)
    X0 = concatenate([MaxPooling2D()(X0),AveragePooling2D()(X0)])
    X0 = Dropout(0.2)(X0)
    ####################
    X1 = Conv2D(filters=128,kernel_size=(5, 2),activation='relu',padding='same',)(input1+tf.reshape(X4,(-1,60,4,2)))
    X1 = Conv2D(filters=128,kernel_size=(5, 3),activation='relu',padding='same',)(X1)
    X1 = concatenate([MaxPooling2D()(X1),AveragePooling2D()(X1)])
    X1 = Dropout(0.2)(X1)
    #####################
    X2 = Conv2D(filters=128,kernel_size=(7, 3),activation='relu',padding='same',)(input1+tf.reshape(X5,(-1,60,4,2)))
    X2 = Conv2D(filters=128,kernel_size=(7, 3),activation='relu',padding='same',)(X2)
    X2 = concatenate([MaxPooling2D()(X2),AveragePooling2D()(X2)])
    X2 = Dropout(0.2)(X2)
    #####################
    ######################
    X0 = Conv2D(filters=256,kernel_size=(3, 3),activation='relu',padding='same',)(X0)
    X0 = Dropout(0.2)(X0)
    X0 = Conv2D(filters=512,kernel_size=(3, 3),activation='relu',padding='same',)(X0)
    X0 = BatchNormalization()(X0)
    X0 = concatenate([GlobalMaxPooling2D()(X0),GlobalAveragePooling2D()(X0)])
    ##########
    X1 = Conv2D(filters=256,kernel_size=(5, 3),activation='relu',padding='same',)(X1)
    X1 = Dropout(0.2)(X1)
    X1 = Conv2D(filters=512,kernel_size=(5, 3),activation='relu',padding='same',)(X1)
    X1 = BatchNormalization()(X1)
    X1 = concatenate([GlobalMaxPooling2D()(X1),GlobalAveragePooling2D()(X1)])
    #########
    X2 = Conv2D(filters=256,kernel_size=(7, 3),activation='relu',padding='same',)(X2)
    X2 = Dropout(0.2)(X2)
    X2 = Conv2D(filters=512,kernel_size=(7, 3),activation='relu',padding='same',)(X2)
    X2 = BatchNormalization()(X2)
    X2 = concatenate([GlobalMaxPooling2D()(X2),GlobalAveragePooling2D()(X2)])
    #########
    
    X = Dropout(0.3)(concatenate([X0,X1,X2]))
    X = Dense(20, activation='softmax')(X)
    return Model([input1,input2], X)

In [ ]:
kfold = StratifiedKFold(10, shuffle=True ,random_state=123456)

proba_t = np.zeros((16000, 20))
#i = 0
for fold, (xx, yy) in enumerate(kfold.split(x, y)):
    y_ = to_categorical(y, num_classes=20)
    model = Net()
    model.compile(loss='categorical_crossentropy',
                  optimizer=Adam(),
                  metrics=['acc'])
    plateau = ReduceLROnPlateau(monitor="val_acc",
                                verbose=0,
                                mode='max',
                                factor=0.5,
                                patience=10)
    early_stopping = EarlyStopping(monitor='val_acc',
                                   verbose=0,
                                   mode='max',
                                   patience=30)
    checkpoint = ModelCheckpoint('best.h5',
                                 monitor='val_acc',
                                 verbose=1,
                                 mode='max',
                                 save_best_only=True)
    model.fit({'input1':x[xx],'input2':x1[xx]}, y_[xx],
              epochs=300,
              batch_size=32,
              verbose=2,
              shuffle=True,
              validation_data=({'input1':x[yy],'input2':x1[yy]}, y_[yy]),
              callbacks=[plateau, early_stopping, checkpoint])
    #i +=1
    model.load_weights('best.h5')
    proba_t += model.predict({'input1':t,'input2':t1}, verbose=0, batch_size=64)/10.0

In [ ]:
np.save('conv2d_pred.npy', proba_t)